# Análise Exploratória de dados: ENAPE 2021, entendendo a evasão escolar e suas principais causas

## Objetivo principal da análise: 
- Identificar padrões comportamentais de alunos que evadem vs os que continuam
- Gerar um perfil básico de aluno com maior chance de evasão
- Explorar as relações entre fatores educacionais e socioeconomicos
- Desenvolver visualizações para exemplificar exploração

## Perguntas norteadoras: 
- Quais fatores que mais influenciam na evasão escolar?
- Existe um perfil de aluno que tem maior chance de evadir?
- Como utilizar da exploração dos dados para aumentar a aderencia escolar de maneira eficiente?

## Passos tomados na análise:
- Divisão por tipo de variável (auxilia na melhor exploração em geral)
- Entendimento das correlações (norteara toda a análise)
- Análise multivariada focando na variável "_inscrito actual_" (target da análise)
- Criação de perfil de aluno com maior chance de evasão baseado nas informações adquiridas 
- Conclusões finais sobre a influencia de fatores educacionais e socioeconomicos

In [246]:
import pandas as pd

## Análise breve geral e definição de tipos para análise

In [247]:
df = pd.read_csv("../data/processed/df_res.csv")
df_complete = pd.read_csv("../data/processed/df_cleaned.csv")

### Definindo tipos

In [248]:
variable_types = {

    'numeric_continuous': [
        'EDAD',
        't_horas',
        'q_hombres',
        'q_mujeres',
        'i_confidence_edu',
        'i_t_tec',
        'i_mental_h',
        'i_tec',
        'i_m_trad',
        'i_m_tech',
    ],

    'binary': [
        'InternetF',
        'inscrito',
        'inscrito_actual',
        'ev_digital',
        'ev_trad',
        'ev_outro',
        'entr_cuidado',
        'apoio',
        'med_virtual',
        'med_presencial',
        'asesorias',
        'extraord',
        'recursado',
        'terminado'
    ],

    'categorical_encoded': [
        'SEXO',
        'nivel_edu',
        'max_nivel_esp',
        'nt_causa',
        'nins_causa',
    ]
}


In [249]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15841 entries, 0 to 15840
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SEXO              15841 non-null  float64
 1   EDAD              15841 non-null  int64  
 2   inscrito          15841 non-null  float64
 3   nivel_edu         14818 non-null  float64
 4   terminado         14818 non-null  float64
 5   nt_causa          15841 non-null  float64
 6   asesorias         13461 non-null  float64
 7   extraord          6391 non-null   float64
 8   recursado         6392 non-null   float64
 9   ev_digital        14593 non-null  float64
 10  inscrito_actual   15841 non-null  float64
 11  nins_causa        1591 non-null   float64
 12  med_presencial    14239 non-null  float64
 13  t_horas           15834 non-null  float64
 14  max_nivel_esp     15841 non-null  float64
 15  q_hombres         15841 non-null  int64  
 16  q_mujeres         15841 non-null  int64 

## Entendendo Correlações 

In [250]:
import plotly.express as px

geralcorr = variable_types['binary'] + variable_types['numeric_continuous']


corr = df[geralcorr].corr()


fig = px.imshow(
    corr,
    text_auto='.4f',          
    aspect='auto',            
    title='Matriz de Correlação',
    color_continuous_scale='RdBu'
)

fig.update_layout(
    width=1200,
    height=1000,
    margin=dict(l=100, r=100, t=100, b=100),
)

fig.show()


### Análise de Correlações

O objetivo aqui é usar a matriz de correlação para ter uma visão geral das variáveis que mais parecem influenciar a evasão de alunos, tomando como referência a variável `inscrito_actual`.

**Correlações de interesse:**
* `inscrito_actual` vs. (`inscrito`, `terminado`, `ev_trad`, `entr_cuidado`, `med_virtual`, `EDAD`, `i_t_tec`, `i_m_trad`, `i_m_tech`)


### Insights e Direcionamento da Análise

Após a análise da matriz de correlação, alguns pontos se destacam e podem guiar os próximos passos.

* **Idade (`EDAD`): A única correlação negativa**
    * **Observação:** A idade (`EDAD`) tem uma correlação negativa com `inscrito_actual`.
    * **Interpretação:** Isso sugere que, quanto mais velho o aluno, maior a sua propensão a evadir. A lógica é que uma relação negativa com 

* **Mídia Virtual (`med_virtual`: 0.61): A correlação positiva mais forte**
    * **Observação:** Uma correlação positiva forte.
    * **Interpretação:** A utilização de plataformas e mídias virtuais está fortemente associada à permanência do aluno no curso.
    * **Ponto de Atenção:** É crucial investigar essa variável mais a fundo. Pode haver um viés aqui.

* **Avaliação Tradicional (`ev_trad`: 0.44): Uma relação moderada**
    * **Observação:** Correlação positiva moderada.
    * **Interpretação:** A forma como as avaliações tradicionais são conduzidas parece ter uma associação relevante com a retenção. Uma hipótese é que métodos já conhecidos pelos alunos podem ajudar na aderência, talvez por não exigirem o uso de materiais que o aluno não possui.

* **Cuidado na Entrada (`entr_cuidado`: 0.38):**
    * **Observação:** Correlação positiva moderada.
    * **Interpretação:** Oferecer algum tipo de cuidado ou suporte especial na entrada do aluno parece aumentar sua aderência ao curso, sugerindo que intervenções iniciais são importantes.

* **Índices em geral (`i_t_tec`, `i_m_trad`, etc.):**
    * **Observação:** Os índices relacionados a materiais e uso de tecnologia também se mostraram positivamente correlacionados.
---

### Pontos Críticos a Considerar

1.  **Correlação não implica Causalidade:** Uma correlação forte apenas indica que duas variáveis se movem juntas, mas não prova que uma causa a outra. Todas as interpretações acima são, por enquanto, hipóteses.

2.  **Risco de Vazamento de Dados (Data Leakage):** Variáveis que só são computadas quando não há evasão são perigosas. Incluir estas variáveis em um modelo preditivo resultaria em uma performance artificialmente alta, mas o modelo seria inútil na prática, pois ele estaria usando informação do futuro para prever o presente. **Variáveis como essa devem ser removidas da modelagem.**

In [251]:
import numpy as np
import pingouin as pg

pvals = corr.corr(method = lambda x, y: pg.corr(x, y)["p-val"].iloc[0])
np.fill_diagonal(pvals.values, np.nan)

fig = px.imshow(
    pvals,
    text_auto='.4f',
    color_continuous_scale='RdBu_r'
    )

fig.update_layout(
    width=1300,
    height=1100,
    margin=dict(l=100, r=100, t=100, b=100),

)

fig

In [254]:
def check_significance(pvals, alpha=0.05):
  n = len(pvals.columns)
  for i in range(n):
      for j in range(i + 1, n):
          if pvals.iloc[i, j] < alpha:
              var1 = pvals.columns[i]
              var2 = pvals.columns[j]
              print(f"Correlação significativa entre {var1} e {var2} (p-valor = {pvals.iloc[i, j]:.4f})")

check_significance(pvals)

Correlação significativa entre InternetF e i_tec (p-valor = 0.0000)
Correlação significativa entre inscrito e inscrito_actual (p-valor = 0.0000)
Correlação significativa entre inscrito e ev_trad (p-valor = 0.0000)
Correlação significativa entre inscrito e apoio (p-valor = 0.0352)
Correlação significativa entre inscrito e med_virtual (p-valor = 0.0052)
Correlação significativa entre inscrito e extraord (p-valor = 0.0262)
Correlação significativa entre inscrito e recursado (p-valor = 0.0159)
Correlação significativa entre inscrito e terminado (p-valor = 0.0000)
Correlação significativa entre inscrito e EDAD (p-valor = 0.0419)
Correlação significativa entre inscrito_actual e ev_trad (p-valor = 0.0002)
Correlação significativa entre inscrito_actual e entr_cuidado (p-valor = 0.0031)
Correlação significativa entre inscrito_actual e apoio (p-valor = 0.0004)
Correlação significativa entre inscrito_actual e med_virtual (p-valor = 0.0000)
Correlação significativa entre inscrito_actual e EDAD (p-

Correlação significativa entre inscrito_actual e ev_trad (p-valor = 0.0002)
Correlação significativa entre inscrito_actual e entr_cuidado (p-valor = 0.0031)
Correlação significativa entre inscrito_actual e apoio (p-valor = 0.0004)
Correlação significativa entre inscrito_actual e med_virtual (p-valor = 0.0000)
Correlação significativa entre inscrito_actual e EDAD (p-valor = 0.0012)
Correlação significativa entre inscrito_actual e t_horas (p-valor = 0.0142)
Correlação significativa entre inscrito_actual e i_t_tec (p-valor = 0.0028)
Correlação significativa entre inscrito_actual e i_m_trad (p-valor = 0.0004)
Correlação significativa entre inscrito_actual e i_m_tech (p-valor = 0.0028)

usando alpha = 0.05, essas foram as variáveis mais críticas encontradas.

In [255]:
import scipy.stats as stats 
important = variable_types['categorical_encoded']

catImportant = df[important]


def anova(df, num_col, cat_col):

    grouped = df.groupby(cat_col)[num_col]

    groups = [group.values[~pd.isnull(group.values)] for _, group in grouped]

    f_val, p_val = stats.f_oneway(*groups)
    return p_val

results = []

def testing_anova(df, categorical_columns, numerical_cols):

  for cat_col in categorical_columns:
    if len(df[cat_col].unique()) > 2:
      for num_col in numerical_cols:
          p_val = anova(df, num_col, cat_col)

          if p_val is not None and p_val < 0.05:
              print(f"Significativo: {num_col} e {cat_col} (p-valor={p_val:.6f})")
              results.append((cat_col, num_col, p_val))

testing_anova(df, catImportant, geralcorr)



Significativo: InternetF e nivel_edu (p-valor=0.000000)
Significativo: inscrito_actual e nivel_edu (p-valor=0.000000)
Significativo: ev_digital e nivel_edu (p-valor=0.000003)
Significativo: ev_trad e nivel_edu (p-valor=0.000000)
Significativo: ev_outro e nivel_edu (p-valor=0.000000)
Significativo: entr_cuidado e nivel_edu (p-valor=0.010134)
Significativo: apoio e nivel_edu (p-valor=0.000000)
Significativo: med_presencial e nivel_edu (p-valor=0.000000)
Significativo: terminado e nivel_edu (p-valor=0.000000)
Significativo: EDAD e nivel_edu (p-valor=0.000000)
Significativo: t_horas e nivel_edu (p-valor=0.000000)
Significativo: q_hombres e nivel_edu (p-valor=0.000000)
Significativo: q_mujeres e nivel_edu (p-valor=0.000036)
Significativo: i_t_tec e nivel_edu (p-valor=0.000000)
Significativo: i_mental_h e nivel_edu (p-valor=0.000021)
Significativo: i_tec e nivel_edu (p-valor=0.000000)
Significativo: i_m_trad e nivel_edu (p-valor=0.000000)
Significativo: i_m_tech e nivel_edu (p-valor=0.000000

c:\Users\Artur\Documents\Artur\DsSchoolDropout\deep_analysis_of_school_dropout_in_mexico\.venv\lib\site-packages\scipy\stats\_axis_nan_policy.py:586: ConstantInputWarning:

Each of the input arrays is constant; the F statistic is not defined or infinite

C:\Users\Artur\AppData\Local\Temp\ipykernel_23132\1788888981.py:13: SmallSampleWarning:

One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.



Significativo: extraord e nins_causa (p-valor=0.000119)
Significativo: recursado e nins_causa (p-valor=0.000019)
Significativo: terminado e nins_causa (p-valor=0.000003)
Significativo: EDAD e nins_causa (p-valor=0.000000)
Significativo: i_tec e nins_causa (p-valor=0.000000)


Significativo: inscrito_actual e nivel_edu (p-valor=0.000000)

Significativo: inscrito_actual e max_nivel_esp (p-valor=0.000000)

Significativo: inscrito_actual e nt_causa (p-valor=0.000000)



In [256]:
from scipy.stats import chi2_contingency


cramers_matrix = pd.DataFrame(
    np.nan,
    index=important,
    columns=important
)
important += ['inscrito_actual', 'ev_trad', 'ev_outro', 'apoio', 'asesorias', 'recursado', 'terminado'] 
np.fill_diagonal(cramers_matrix.values, 1.0)


def cramers_v(contingency_table):
    chi2, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    phi2 = chi2 / n
    r, k = contingency_table.shape
    return np.sqrt(phi2 / min(k-1, r-1))


for i, col1 in enumerate(important):
    for j, col2 in enumerate(important):
        if i < j:
            contingency = pd.crosstab(
                df[col1],
                df[col2]
            )
            cv = cramers_v(contingency.values)
            cramers_matrix.loc[col1, col2] = cv
            cramers_matrix.loc[col2, col1] = cv

fig = px.imshow(
    cramers_matrix,
    text_auto='.4f',
    color_continuous_scale='RdBu_r'
    )

fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=100, r=100, t=100, b=100),

)

fig



C:\Users\Artur\AppData\Local\Temp\ipykernel_23132\3358702205.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Artur\AppData\Local\Temp\ipykernel_23132\3358702205.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Artur\AppData\Local\Temp\ipykernel_23132\3358702205.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Artur\AppData\Local\Temp\ipykernel_23132\3358702205.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Artur\AppData\Local\Temp\ipykernel_23132\3358702205.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Artur\AppData\Local\Temp\ipykernel_23132\3358702205.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Artur\AppData\Local\Temp\ipykernel_23132\3358702205.py:18: RuntimeWarning:

invalid value encountered in scalar divide



In [257]:
percentual_evasores = df['inscrito_actual'].value_counts(normalize=True) * 100

percentual_evasores

inscrito_actual
1.0    89.956442
0.0    10.043558
Name: proportion, dtype: float64

In [258]:
for i in geralcorr:
    table = df.groupby("inscrito_actual")[i]
    display(i)
    display(table.describe())


'InternetF'

,count,mean,std,min,25%,50%,75%,max
inscrito_actual,,,,,,,,
0.0,1591.0,0.510999,0.500036,0.0,0.0,1.0,1.0,1.0
1.0,14250.0,0.720842,0.448601,0.0,0.0,1.0,1.0,1.0


'inscrito'

,count,mean,std,min,25%,50%,75%,max
inscrito_actual,,,,,,,,
0.0,1591.0,0.494657,0.500129,0.0,0.0,0.0,1.0,1.0
1.0,14250.0,0.984632,0.123017,0.0,1.0,1.0,1.0,1.0


'inscrito_actual'

...

,count,mean,std,min,25%,50%,75%,max
inscrito_actual,,,,,,,,
0.0,1591.0,0.291640,0.454661,0.0,0.0,0.0,1.0,1.0
1.0,14250.0,0.507439,0.499962,0.0,0.0,1.0,1.0,1.0


'i_m_trad'

,count,mean,std,min,25%,50%,75%,max
inscrito_actual,,,,,,,,
0.0,1591.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
1.0,14250.0,0.71986,0.449083,0.0,0.0,1.0,1.0,1.0


'i_m_tech'

,count,mean,std,min,25%,50%,75%,max
inscrito_actual,,,,,,,,
0.0,1591.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1.0,14250.0,0.589474,0.398249,0.0,0.5,0.5,1.0,1.0


In [259]:
def plot_porcentagem_inscritos(df, colunas, target, nomes_colunas=None):

    dados_plot = []

    for col in colunas:
        nome_exibicao = nomes_colunas[col] if nomes_colunas and col in nomes_colunas else col

        porcentagens = df.groupby(col)[target].mean() * 100

        for valor, porcentagem in porcentagens.items():
            dados_plot.append({
                'variavel': nome_exibicao,
                'valor_binario': 'Sim' if valor == 1 else 'Não',
                'porcentagem_inscritos': porcentagem
            })

    df_plot = pd.DataFrame(dados_plot)

    fig = px.bar(df_plot,
                 x='variavel',
                 y='porcentagem_inscritos',
                 color='valor_binario',
                 barmode='group',
                 labels={
                     'porcentagem_inscritos': 'Porcentagem de inscritos',
                     'variavel': 'Variável analisada',
                     'valor_binario': 'Valor'
                 },
                 title='Porcentagem de inscritos por variável')

    fig.update_layout(yaxis_tickformat=".1f")
    return fig


cols = ['ev_digital', 'ev_trad', 'ev_outro']
d = {
    'ev_digital': 'Avaliação digital',
    'ev_trad': 'Avaliação tradicional',
    'ev_outro': 'Outro tipo de avaliação'
}

fig = plot_porcentagem_inscritos(df, cols, 'inscrito_actual', d)
fig.show()



In [260]:
cols = ['med_virtual', 'med_presencial']
d = {
    'med_virtual': 'Plataformas digitais usadas na classe',
    'med_presencial': 'Atividades entregues presencialmente', #por ter dado 100%, provavlemente isso só foi avaliado se a pessoa não evadiu

}
fig = plot_porcentagem_inscritos(df, cols, 'inscrito_actual', d)
fig.show()

In [261]:
educations = [ 'extraord', 'recursado', 'terminado', 'apoio', 'entr_cuidado']
renameEducations = {
    'extraord': 'Fez prova de recuperação',
    'recursado': 'teve que refazer alguma disciplina',
    'terminado': 'terminou o último ano letivo', #chance de algume voltar a estudar é baixissima!
    'apoio': 'Recebeu apoio nos estudos (familiar ou não)', #mais comum em menores idades
    'entr_cuidado': 'Teve algum tipo de cuidado especial ao entrar na escola',
}
fig = plot_porcentagem_inscritos(df, educations, 'inscrito_actual', d )
fig

In [262]:
#analise de fatores socieconômicos
#flag de indices, se maior que 0.5 1 se não 0

def flag_indices(df, cols):
    for i in cols:
        df[f'flag_{i}'] = (df[i] > 0.5).astype(int)
    return df

df = flag_indices(df, ['i_confidence_edu', 'i_tec'])
"""
    'binary': [
        'InternetF',
        'inscrito',
        'inscrito_actual',
        'ev_digital',
        'ev_trad',
        'ev_outro',
        'entr_cuidado',
        'apoio',
        'med_virtual',
        'med_presencial',
        'asesorias',
        'extraord',
        'recursado',
        'terminado'
    ],
"""

cols = ['InternetF' , 'flag_i_confidence_edu', 'flag_i_tec']
d = {
    'InternetF': 'Acesso fixo à Internet',
    'entr_cuidado': 'Cuidado com crianças durante o estudo',
    'flag_i_confidence_edu': 'Confia na educação', #pouca importancia se confia ou não na educação, demosntra que evadir ou não pode estar ligado a fatores externos
    'flag_i_tec': 'Aparelhos eletronicos em casa'
}

fig = plot_porcentagem_inscritos(df, cols, 'inscrito_actual', d)
fig.show()


In [263]:
import plotly.express as px

def continuous_discretize_plot(df, col, target, nbins,  renames = None):

    nameNewCol = f"{col}_binned"
    df[nameNewCol] = pd.cut(df[col], nbins)

    grouped = df.groupby(nameNewCol)[target].mean() * 100
    grouped = grouped.reset_index()
    grouped[nameNewCol] = grouped[nameNewCol].astype(str)

    if renames:
        grouped[nameNewCol] = grouped[nameNewCol].replace(renames)
        
    fig = px.bar(grouped, 
                x=nameNewCol, 
                y=target, 
                color='inscrito_actual', 
                color_continuous_scale='Reds_r',  
                labels={nameNewCol: col, target: f'% {target}'}, 
                title=f'Taxa de {target} por {col}')
    
    fig.update_yaxes(tickformat=".2f")

    return fig


novasFaixasEtarias = {
    "(5.988, 9.0]": "6–9 anos",
    "(9.0, 12.0]": "10–12 anos",
    "(12.0, 15.0]": "13–15 anos",
    "(15.0, 18.0]": "16–18 anos"
}


fig = continuous_discretize_plot(df, 'EDAD', 'inscrito_actual', 4, renames=novasFaixasEtarias)
fig.update_layout(title="Taxa de aderência escolar por faixa etária")
fig.show()





C:\Users\Artur\AppData\Local\Temp\ipykernel_23132\2362288079.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [264]:
#cheando se os dados batem
display(df['nins_causa'].notna().sum())
display((df['inscrito_actual'] == 0).sum())

np.int64(1591)

np.int64(1591)

In [273]:
causes_nins_short_dict = {
    1: "Financial reasons",
    2: "Work obligations",
    3: "Lack of interest",
    4: "Completed education",
    5: "Poor academic performance",
    6: "School access issues",
    7: "Disability",
    8: "COVID-19 pandemic",
    9: "COVID-19 illness",
    10: "Health problems (non-COVID)",
    11: "Marriage/Pregnancy",
    12: "Household/Care duties",
    13: "Family opposition",
    14: "Personal conflicts",
    15: "Other",
    16: "Considered too young"
}
df['faixa_etaria'] = df['EDAD_binned']
principaisCausas = df[['faixa_etaria', 'nins_causa']].groupby(['faixa_etaria', 'nins_causa']).size().reset_index(name='qtd_tot')

principaisCausas['nins_causa'] = principaisCausas['nins_causa'].replace(causes_nins_short_dict)

principaisCausas['faixa_etaria'] = principaisCausas['faixa_etaria'].astype(str)
principaisCausas['faixa_etaria'] = principaisCausas['faixa_etaria'].replace(novasFaixasEtarias)

principaisCausas['total_relativo'] = principaisCausas.groupby('faixa_etaria')['qtd_tot'].transform('sum')
principaisCausas['percentual'] = (principaisCausas['qtd_tot'] / principaisCausas['total_relativo']) * 100

fig = px.sunburst(
    principaisCausas,
    path=['faixa_etaria', 'nins_causa'],  
    values='qtd_tot',
    title='Reasons for Not Attending School by Age Group'
)


fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=100, r=100, t=100, b=100),

)
fig.write_html("sunburst_chart_corrected.html")

fig

C:\Users\Artur\AppData\Local\Temp\ipykernel_23132\1404854243.py:20: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [268]:
#aderencia relacionada a nivel de educação

remapNivelEdu = {
    1: "Early Childhood Education (Daycare/Nursery)",
    2: "Preschool",
    3: "Elementary School",
    4: "Middle School",
    5: "Technical Professional  (after Middle School)",
    6: "High School (General)",
    7: "Technical High School (Vocational High School)",
    8: "Technical Associate's Degree after High School",
    9: "Bachelor's Degree",
    10: "Specialization/Graduate Certificate",
    11: "Master's Degree",
    12: "Doctorate"
}

niveledu = df[['nivel_edu', 'inscrito_actual']].groupby(['nivel_edu', 'inscrito_actual']).size().reset_index(name='qtd_tot')
niveledu['nivel_edu'] = niveledu['nivel_edu'].replace(remapNivelEdu)
niveledu['total_relativo'] = niveledu.groupby('nivel_edu')['qtd_tot'].transform('sum')
niveledu['percentual'] = (niveledu['qtd_tot'] / niveledu['total_relativo']) * 100
fig = px.sunburst(
    niveledu,
    path=['nivel_edu', 'inscrito_actual'],  
    values='percentual',
    title='School Enrollment by Education Level'
)

fig.update_layout(
    width=1500,
    height=1200,
    margin=dict(l=100, r=100, t=100, b=100),

)

fig